In [ ]:
import sys
!{sys.executable} -m pip install -r requirements.txt

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
import cvxpy as cvx
import plotly as py
import plotly.graph_objs as go
import helper

py.offline.init_notebook_mode(connected=True)
%matplotlib inline
plt.style.use('ggplot')

## 通过 Alpha 模型和风险模型进行优化
在这道练习中，我们将使用包含 3 支股票的股票池构建一个优化问题。这个问题描述的是一种虚拟情形，而股票池包含 3 支股票使我们能够绘制三维图形。有助于描绘每一步发生的情况，并帮助你理解问题。这道练习包括以下步骤：
1. 根据股价数据创建一个基于 1 年动量的 alpha 向量。
2. 使用主成分分析创建一个风险模型。
3. 使用 alpha 向量和风险模型创建优化问题，并应用一组标准约束条件。
4. 解决优化问题。


## 加载数据
从文件 `stock_prices_advanced_optimization.csv` 中加载数据。

In [ ]:
prices = pd.read_csv('stock_prices_advanced_optimization.csv', parse_dates=['date'], index_col=0)

数据是以下 3 支股票从 2013 年到 2017 年这 4 年的股价走势：股票 A、股票 B 和股票 C。

In [ ]:
prices.head()

In [ ]:
prices.plot()

计算这些股价数据的收益率。

In [ ]:
rets = prices.pct_change()[1:].fillna(0)

## 创建 alpha 向量

In [ ]:
from scipy.stats import zscore

我们将根据一年的动量创建 alpha 向量。

In [ ]:
# 1-yr momentum alpha

def log_returns(series):
    return np.log(series[-1])-np.log(series[0])
    
alpha = prices.rolling(window=252).apply(log_returns).rank(axis='columns').apply(zscore, axis='columns')

我们将最近的 alpha 值行作为 alpha 向量。每支股票都应该在向量中对应一个值。

In [ ]:
# Take most recent set of values
alpha_vector = alpha.iloc[-1,:]
print(alpha_vector)

对于此问题，我们将使用优化目标 $-\boldsymbol{\alpha}^T \mathbf{x}$。注意，我们要_最小化_此函数（以便最大化 alpha）。我们绘制 $-\boldsymbol{\alpha}^T \mathbf{x}$ 函数的图形，其中 $\mathbf{x}$ 是自变量，从而更好地了解函数的“形状”。

In [ ]:
n = 10
x = y = z = np.linspace(-2,2,n)

xv, yv, zv = np.meshgrid(x, y, z, indexing='ij')
obj_val = np.full(xv.shape, np.nan)

for i in range(n):
    for j in range(n):
        for k in range(n):
            obj_val[i,j,k] = -alpha_vector[0]*xv[i,j,k]-alpha_vector[1]*yv[i,j,k]-alpha_vector[2]*zv[i,j,k]

In [ ]:
hover_text = helper.generate_hover_text(xv, yv, zv, 'Weight on Stock A', 'Weight on Stock B', 'Weight on Stock C', fcn_values=obj_val, fcn_label='Objective Function')

trace1 = go.Scatter3d(
    x=xv.flatten(),
    y=yv.flatten(),
    z=zv.flatten(),
    text = hover_text.flatten(),
    mode='markers',
    marker=dict(
        size=4,
        color=obj_val.flatten(),     # set color to an array/list of desired values
        colorscale='Viridis',   # choose a colorscale
        colorbar=dict(
                title='Objective Function'
            ),
        opacity=0.4,
        showscale=True
    ),
    hoverinfo = 'text'
)

data = [trace1]
layout = helper.create_standard_layout('Alpha Vector Optimization Objective Function', 'Weight on')
fig = go.Figure(data=data, layout=layout)
py.offline.iplot(fig)

在此图形中，目标函数的值用权重空间中的每个点的颜色表示。注意函数随着股票 C 的权重而增大，并随着股票 B 的权重而减小，但是不依赖于股票 A 的权重。

如果我们现在就运行优化，尝试最小化目标函数，但是没有任何约束条件，会发生什么？

In [ ]:
def find_optimal_holdings(alpha_vector):
    """
    Use cvxpy to construct and solve an optimization problem. Use -alpha*x as the objective, and use no constraints.

    Parameters
    ----------
    alpha_vector : DataFrame
        The 3-stock alpha vector calculated above.

    Returns
    -------
    optimal_weights : DataFrame
        A DataFrame containing the optimal weights calculated by the optimization algorithm.
    """
    #TODO: Implement function
    
    return None

In [ ]:
optimal_weights = find_optimal_holdings(
    alpha_vector
)
print("Optimal weights: ", optimal_weights)

不出所料，如果没有约束条件，问题不受控制。我们可以通过对股票 C 采取越来越大的做多头寸，并对股票 B 采取越来越大的做空头寸，获得越来越小的目标函数值。但是这样会提高风险和杠杆比率。这时候约束条件就派上用场了。

## 创建风险模型

我们将针对这 3 支股票创建一个基于 PCA 的风险模型。首先绘制收益率数据及其均值，看看数据的形状如何。

In [ ]:
m = rets.mean()

In [ ]:
# Trace for mean return vector m
mean_vec = np.vstack((np.full(3, 0), m.values)).T

hover_text2 = helper.generate_hover_text(mean_vec[0], mean_vec[1], mean_vec[2], 'Mean of Returns of Stock A', 'Mean of Returns of Stock B', 'Mean of Returns of Stock C', sig_figs = 7)

trace2 = go.Scatter3d(
    x=mean_vec[0],
    y=mean_vec[1],
    z=mean_vec[2],
    mode='lines+markers',
    marker=dict(
        size=4,
        color='red',
        opacity=0.9,
        symbol="diamond"
    ),
    name = 'mean daily return',
    text = hover_text2.flatten(),
    hoverinfo = 'text'
)


# Trace for data
hover_text3 = helper.generate_hover_text(rets['A'].values, rets['B'].values, rets['C'].values, 'Return of Stock A', 'Return of Stock B', 'Return of Stock C')

trace3 = go.Scatter3d(
    x=rets['A'].values,
    y=rets['B'].values,
    z=rets['C'].values,
    mode='markers',
    marker=dict(
        size=4,
        color='#7FB3D5',  
        opacity=0.3,
    ),
    name = 'daily returns',
    text = hover_text3.flatten(),
    hoverinfo = 'text'
)

data = [trace2, trace3]

layout = helper.create_standard_layout('Returns Data', 'Return of')
fig = go.Figure(data=data, layout=layout)
py.offline.iplot(fig)

需要注意几点。
1. 与数据的波动性相比，均值收益率向量很小。没错，这是一个采用虚拟数据的虚拟示例，但是通常也是这样。
2. 股票 C 的波动性大于股票 B 的波动性，股票 B 的波动性大于股票 A 的波动性。

根据这些数据，我们已经能够预期 PCA 将是什么样的结果。我们期望第一个主成分指向数据的最大方差。根据股票 C 的波动性，我们预期第一个主成分的方向与“股票 C 轴”大致平行。如果结果截然不同，我们就知道哪里出错了。下面运行 PCA 算法，看看结果如何。

首先均值居中数据。

In [ ]:
rets = rets - m

然后创建一个拟合风险模型的类。

In [ ]:
from sklearn.decomposition import PCA

class RiskModelPCA():
    
    ANN_FACTOR = 252
    
    def __init__(self, num_factors):
        self._num_factors = num_factors
        self.num_stocks_ = None
        self.factor_betas_ = None
        self.factor_returns_ = None
        self.common_returns_ = None
        self.residuals_ = None
        self.factor_cov_matrix_ = None
        self.idio_var_matrix_ = None
        self.explained_variance_ratio_ = None

    def fit(self, returns):
        self.num_stocks_ = len(returns.columns)
        mod = PCA(n_components=self._num_factors, svd_solver='full')
        mod.fit(returns)
        
        self.factor_betas_ = pd.DataFrame(
            data=mod.components_.T,
            index=returns.columns
        )
        
        self.factor_returns_ = pd.DataFrame(
            data=mod.transform(rets),
            index=returns.index
        )
        
        self.explained_variance_ratio_ = mod.explained_variance_ratio_
        
        self.common_returns_ = pd.DataFrame(
            data=np.dot(self.factor_returns_, self.factor_betas_.T),
            index=returns.index
        )
        self.common_returns_.columns = returns.columns
        
        self.residuals_ = (returns - self.common_returns_)
        
        self.factor_cov_matrix_ = np.diag(
            self.factor_returns_.var(axis=0, ddof=1)*RiskModelPCA.ANN_FACTOR
        )
        
        self.idio_var_matrix_ = pd.DataFrame(
            data=np.diag(np.var(self.residuals_))*RiskModelPCA.ANN_FACTOR,
            index=returns.columns
        )
        
        self.idio_var_vector_ = pd.DataFrame(
            data=np.diag(self.idio_var_matrix_.values),
            index=returns.columns
        )
        
        self.idio_var_matrix_.columns = index=returns.columns

    def get_factor_exposures(self, weights):
        B = self.factor_betas_.loc[weights.index]
        return B.T.dot(weights)

用 2 个因子（即保留 2 个主成分）拟合风险模型。

In [ ]:
rm = RiskModelPCA(2) # create an instance of the class with 2 factors
rm.fit(rets) # fit the model on our data

查看因子（主成分）

In [ ]:
rm.factor_betas_

第一个主成分几乎完全指向“股票 C”的方向。第二个主成分几乎完全指向“股票 B”的方向。这很合理，因为“股票 C”的方向方差最大，除去该方差后，接下来“股票 B”的方向方差最大。下面我们绘制主成分向量，看得更清晰。

In [ ]:
PC_scaler = 0.04 # The PC vectors have length 1, but this is larger than the scale of our data, so for visualization purposes, let's plot scaled-down versions of the PCs. 

# Trace for PC 0
pc0 = np.vstack((np.full(3, 0), rm.factor_betas_[0].values)).T*PC_scaler

hover_text4 = helper.generate_hover_text(pc0[0], pc0[1], pc0[2], 'Return of Stock A', 'Return of Stock B', 'Return of Stock C')

trace4 = go.Scatter3d(
    x=pc0[0],
    y=pc0[1],
    z=pc0[2],
    mode='lines+markers',
    marker=dict(
        size=4,
        color='#45B39D',
        opacity=0.9,
        symbol="diamond"
    ),
    line=dict(
        color='#45B39D',
        width=3
    ),
    name = 'PC 0',
    text = hover_text4.flatten(),
    hoverinfo = 'text'

)

# Trace for PC 1
pc1 = np.vstack((np.full(3, 0), rm.factor_betas_[1].values)).T*PC_scaler

hover_text5 = helper.generate_hover_text(pc1[0], pc1[1], pc1[2], 'Return of Stock A', 'Return of Stock B', 'Return of Stock C')

trace5 = go.Scatter3d(
    x=pc1[0],
    y=pc1[1],
    z=pc1[2],
    mode='lines+markers',
    marker=dict(
        size=4,
        color='#FFC300',
        opacity=0.9,
        symbol="diamond"
    ),
    line=dict(
        color='#FFC300',
        width=3
    ),
    name = 'PC 1',
    text = hover_text5.flatten(),
    hoverinfo = 'text'

)

# Trace for data
hover_text6 = helper.generate_hover_text(rets['A'].values, rets['B'].values, rets['C'].values, 'Return of Stock A', 'Return of Stock B', 'Return of Stock C')

trace6 = go.Scatter3d(
    x=rets['A'].values,
    y=rets['B'].values,
    z=rets['C'].values,
    mode='markers',
    marker=dict(
        size=4,
        color='#7FB3D5', 
        opacity=0.3,
    ),
    name = 'daily returns',
    text = hover_text6.flatten(),
    hoverinfo = 'text'
)

data = [trace4, trace5, trace6]

layout = helper.create_standard_layout('Returns Data with Factor (PC) Directions', 'Return of')
fig = go.Figure(data=data, layout=layout)
py.offline.iplot(fig)

我们看看保留的两个因子解释的方差部分。应该与我们认为的前两个因子解释的方差相对百分比一致。

In [ ]:
plt.bar(np.arange(2), rm.explained_variance_ratio_, color=['#45B39D', '#FFC300']);
plt.title('% of Variance Explained by Each Factor');

下面看看因子收益率。我们将它们转换为“价格序列”，并可视化它们随时间推移的变化。它们是用“因子”(PC) 基点（数据向量到因子维度的投射值）解释的数据。

In [ ]:
(rm.factor_returns_ + 1).cumprod().plot(color=['#45B39D', '#FFC300'])

注意，在此示例中，因为第一个因子非常接近股票 C 的收益率方向，所以因子 0 的因子收益率（已转换为“价格序列”）看起来很像股票 C 的价格变化，但是颠倒的。这是因为主成分是表示空间中方向的向量，它们仅在信号变化方面是唯一的，所以它们的投射值在信号变化方面是唯一的。

In [ ]:
prices.plot()

## 创建优化约束条件
正如在之前看到的，我们需要对我们的优化问题加以约束。下面创建并绘制以下约束条件：
* 基于风险模型的风险约束条件
* 杠杆约束条件
* 市场中性约束条件
* 因子暴露度约束条件
* 个别权重约束条件

#### 风险

In [ ]:
B = rm.factor_betas_
F = rm.factor_cov_matrix_
S = np.diag(rm.idio_var_vector_.values.flatten())

我们使用 $\mathbf{B}$、$\mathbf{F}$ 和 $\mathbf{S}$ 矩阵写一个函数，它会接受投资组合权重并输出投资组合风险。投资组合风险的公式为 $\mathbf{x}^T(\mathbf{B}^T\mathbf{F}\mathbf{B} + \mathbf{S})\mathbf{x}$，但是注意所有矩阵的方向都要正确。

In [ ]:
def risk_fcn(x):
    """
    Calculate portfolio risk.

    Parameters
    ----------
    x : numpy array
        The vector of portfolio weights.

    Returns
    -------
    risk : float
        Portfolio risk.
    """
    #TODO: Implement function
    
    return None

In [ ]:
n_samples = 25
grid_max = 2.5
risk_grid, spacing, xv, yv, zv = helper.evaluate_fcn_on_grid(grid_max, n_samples, risk_fcn)

下面我们看看投资组合风险在三维空间里相对于投资组合权重的函数图。

In [ ]:
hover_text = helper.generate_hover_text(xv, yv, zv, 'Weight on Stock A', 'Weight on Stock B', 'Weight on Stock C', fcn_values=risk_grid, fcn_label='Portfolio Risk')

trace7 = go.Scatter3d(
    x=xv.flatten(),
    y=yv.flatten(),
    z=zv.flatten(),
    mode='markers',
    marker=dict(
        size=4,
        color=risk_grid.flatten(), 
        colorscale='Viridis',
        opacity=0.3,
        showscale=True
    ),
    text = hover_text.flatten(),
    hoverinfo = 'text'
)

data = [trace7]
layout = helper.create_standard_layout('Portfolio Risk as Modeled by our PCA Risk Model', 'Weight on')
fig = go.Figure(data=data, layout=layout)
py.offline.iplot(fig)

这个图表在三维空间里的多个点上标注了颜色，表示投资组合风险相对于每支股票权重的函数。注意，它会随着每支股票的权重而增大。这样有助于我们可视化风险函数，但是约束风险后看起来如何？我们可视化风险低于某个上限值的区域边界。这样有助于可视化我们要在优化问题中搜寻的空间形状。每个约束条件都操作一遍。

In [ ]:
risk_data = helper.plot_iso_surface(risk_grid, 0.05, 25, 2.5, "Portfolio Risk = 0.05", '#F5B7B1', True)

#### 杠杆效应

下面编写一个函数，它会接受投资组合权重并输出投资组合杠杆 $\sum_i|x_i|$，使我们能够可视化满足约束条件 $\sum_i|x_i| \leq 1$ 的空间。

In [ ]:
def lev_fcn(x):
    """
    Calculate portfolio leverage.

    Parameters
    ----------
    x : numpy array
        The vector of portfolio weights.

    Returns
    -------
    leverage : float
        Portfolio risk.
    """
    #TODO: Implement function
    
    return None

n_samples = 25
grid_max = 2.5

lev_grid, _, _, _, _ = helper.evaluate_fcn_on_grid(grid_max, n_samples, lev_fcn)
lev_data = helper.plot_iso_surface(lev_grid, 1.0, n_samples, grid_max, "Leverage Ratio = 1", '#1F618D', True)

#### 市场中性

对于市场中性约束条件，我们对权重之和加以约束。我们写一个函数，它会根据投资组合权重计算权重之和。然后我们可视化平面 $\sum_i x_i = 0$。

In [ ]:
def mn_fcn(x):
    """
    Calculate the sum of the portfolio weights.

    Parameters
    ----------
    x : numpy array
        The vector of portfolio weights.

    Returns
    -------
    sum_of_weights : float
        Sum of the portfolio weights.
    """
    #TODO: Implement function
    
    return None

n_samples = 25
grid_max = 2.5

mn_grid, _, _, _, _ = helper.evaluate_fcn_on_grid(grid_max, n_samples, mn_fcn)
mn_data = helper.plot_iso_surface(mn_grid, 0, n_samples, grid_max, "Sum of Weights = 0", '#D35400', True)

#### 因子暴露度

下面计算因子暴露度。然后绘制因子暴露度上限确定的平面。优化问题会要求解处于每对平面之间。我们要求每个因子暴露度介于  -0.1 和 0.1 之间。

In [ ]:
def fac_fcn(x):
    """
    Calculate portfolio factor exposures.

    Parameters
    ----------
    x : numpy array
        The vector of portfolio weights.

    Returns
    -------
    fac_exposures : numpy array
        Portfolio factor exposures.
    """
    #TODO: Implement function
    
    return None

grid_max = 2.5
n_samples = 25

fac_grid, spacing, _, _, _ = helper.evaluate_fcn_on_grid(grid_max, n_samples, fac_fcn)

factor_limit_data = []

for factor in range(B.shape[1]):
    for l in range(2):
        mult_fac = l*2-1
        factor_limit_data.extend(helper.plot_iso_surface(fac_grid[:,:,:,factor], 0.1*mult_fac, n_samples, grid_max, 'Factor Exposure Limits', '#D2B4DE', False))

layout = helper.create_standard_layout('Factor Exposure Limits', 'Weight on')

fig = go.Figure(data=factor_limit_data, layout=layout)         
        
py.offline.iplot(fig)

#### 单个权重限制

最后，我们看看由关于每个权重的约束条件确立的空间。我们要求每个权重介于 -0.55 和 0.55 之间。

In [ ]:
x_max = 0.55

x = np.array([-1, -1, 1, 1, -1, -1, 1, 1])*x_max
y = np.array([-1, 1, 1, -1, -1, 1, 1, -1])*x_max
z = np.array([-1, -1, -1, -1, 1, 1, 1, 1])*x_max
hover_text = helper.generate_hover_text(x, y, z, 'Weight on Stock A', 'Weight on Stock B', 'Weight on Stock C')


weight_data = [
    go.Mesh3d(
        x = x,
        y = y,
        z = z,
        colorscale = '#FCF3CF',
        intensity = np.full(8, 1),
        i = [7, 0, 0, 0, 4, 4, 6, 6, 4, 0, 3, 2],
        j = [3, 4, 1, 2, 5, 6, 5, 2, 0, 1, 6, 3],
        k = [0, 7, 2, 3, 6, 7, 1, 1, 5, 5, 7, 6],
        name='Weight on Stock B',
        showscale=False,
        opacity = 0.3,
        hoverinfo='none'
    )
]

trace = go.Scatter3d(
    x=x,
    y=y,
    z=z,
    mode='markers',
    marker=dict(
        size=6,
        opacity=0.0001,
        color='#BFB1A8', 
    ),
    text = hover_text.flatten(),
    hoverinfo = 'text',
    showlegend=False
)
    
weight_data = [weight_data[0], trace]    
    
layout = helper.create_standard_layout('Individual Weight Limits', 'Weight on')
fig = go.Figure(data=weight_data, layout=layout)
py.offline.iplot(fig)

#### 在相同的坐标轴下绘制所有约束条件

最后，可视化满足所有约束条件的空间。它必须位于所有封闭的空间内，即在市场中性约束条件定义的平面上，并在因子暴露度约束条件定义的平面之间。

In [ ]:
data = risk_data
data.extend(lev_data)
data.extend(mn_data)
data.extend(factor_limit_data)
data.extend(weight_data)
layout = helper.create_standard_layout('Visualize Intersection of All Constraints', 'Weight on')
fig = go.Figure(data=data, layout=layout)
py.offline.iplot(fig)

## 优化
最后，定义并运行包含上述目标函数的优化问题，但是这次施加了我们讨论的所有约束条件。优化流程是要寻找空间中满足所有约束条件并且能最小化 alpha 函数的点。

In [ ]:
def find_optimal_holdings(
    alpha_vector,
    risk_model,
    risk_cap=0.05,
    factor_max=10.0,
    factor_min=-10.0,
    x_max=0.55,
    x_min=-0.55):
    
    """
    Define an optimization problem. It takes in several inputs and optimization parameters and outputs the
    optimized weights. The objective should minimize the objective -alpha*x, but also apply the risk, leverage,
    market neutral, factor exposure, and invidiual weight constraints.

    Parameters
    ----------
    alpha_vector : numpy array
        The alpha vector used in the optimization objective.
    risk_model : RiskModelPCA
        The risk model calculated above using PCA.
    risk_cap : float
        The limit to be placed on portfolio risk.
    factor_max : float
        The factor exposure upper limit.
    factor_min : float
        The factor exposure lower limit.
    x_max : float
        The individual weight upper limit.
    x_min : float
        The individual weight lower limit.

    Returns
    -------
    optimal_weights : numpy array
        The optimal portfolio weights.
    """
    #TODO: Implement function
    
    return None

In [ ]:
optimal_weights = find_optimal_holdings(
    alpha_vector,
    rm
)

In [ ]:
optimal_weights

最佳权重：股票 A 的权重为 0，股票 B 50% 做多权重，股票 C 50% 做空权重。

[这是解答 notebook](Advanced_Opt_solution.ipynb)。